In [2]:
import sys
sys.executable

'/Users/b/bio/cardiac/model1/new_env/bin/python'

In [3]:
# import packages
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import torch
import torch.nn as nn
import torch.optim as optim
import random
import numpy as np
import spacy
import datasets
#import torchtext
import tqdm
import evaluate
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import pyabf
import ast
from datasets import Dataset

In [4]:
# read data
df = pd.read_csv('/Users/b/bio/cardiac/model1/phases.csv')

In [5]:
def get_phase_data(df, index):
    """
    Converts string representations of x and y values at a given index in the DataFrame
    into a list of (x, y) tuples.

    Parameters:
    df (pd.DataFrame): The DataFrame containing the data.
    index (int): The index of the data to extract.

    Returns:
    list: A list of (x, y) tuples.
    """
    phase_x = ast.literal_eval(df['X Values'][index])
    phase_y = ast.literal_eval(df['Y Values'][index])
    phase_data = list(zip(phase_x, phase_y))
    return phase_data



In [6]:
def flatten_tuple_list(tuple_list):
    flat_list = []
    for tup in tuple_list:
        flat_list.extend(tup)
    return flat_list

In [7]:
def round_list_numbers(numbers_list, decimals=5):
    """
    Round all numbers in a list to the specified number of decimal places.

    Args:
    numbers_list (list): The list of numbers.
    decimals (int): The number of decimal places to round to.

    Returns:
    list: A list of rounded numbers.
    """
    return [round(num, decimals) for num in numbers_list]

In [8]:
data_dict = []
index = 0
i = 0
while index < len(df):
    file = df['File Path'][index]
    sweepNum = df['Sweep Number'][index]
    phaseNum = df['Phase'][index]
    abf = pyabf.ABF(file)
    abf.setSweep(sweepNum)
    
    # Extract sweepX and sweepY
    sweepX = abf.sweepX
    sweepY = abf.sweepY
    
    # Create a list of tuples (x, y)
    totalSweep = list(zip(sweepX, sweepY))
    
    phase0 = get_phase_data(df, index)
    phase1 = get_phase_data(df, index + 1)
    phase2 = get_phase_data(df, index + 2)
    phase3 = get_phase_data(df, index + 3)
    phase4 = get_phase_data(df, index + 4)

    data_dict.append({})
    data_dict[i]["input"] = round_list_numbers(flatten_tuple_list(totalSweep))
    data_dict[i]["output"] = round_list_numbers(flatten_tuple_list([phase0[0], phase1[0], phase2[0], phase3[0], phase4[0], phase4[-1]]))

    
    index+=5
    i+=1

In [9]:
lst = [len(data_dict[x]["output"]) for x in range(len(data_dict))]
print(lst)
print(len(lst))

[12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12]
27


In [10]:
import random

def split_data_dict(data_dict, train_ratio=0.8, val_ratio=0.1, test_ratio=0.1):
    # Ensure the ratios add up to 1
    assert train_ratio + val_ratio + test_ratio == 1.0, "Ratios must sum to 1"
    
    total_length = len(data_dict)
    train_index = int(total_length * train_ratio)
    val_index = train_index + int(total_length * val_ratio)
    

    train_data = data_dict[:train_index]
    val_data = data_dict[train_index:val_index]
    test_data = data_dict[val_index:]
    
    return train_data, val_data, test_data


In [11]:
# split dataset 80/10/10

train_data, valid_data, test_data = split_data_dict(data_dict)
print(len(train_data))
print(len(valid_data))
print(len(test_data))

21
2
4


In [12]:
train_data[0]["output"]

[0.025,
 -82.1228,
 0.035,
 54.01611,
 0.1,
 37.20093,
 0.3,
 9.61304,
 0.45,
 -79.07104,
 1.9999,
 -82.51953]

In [17]:
# vocab...

"""
{0: <unk>,
 1: <pad>,
 2: <sos>,
 3: <eos>,
 4: 0.0,
 5: -79.58984,
 6: 0.00005,
 7: -79.55933,
 ...
 }
 """

'\n{0: <unk>,\n 1: <pad>,\n 2: <sos>,\n 3: <eos>,\n 4: 0.0,\n 5: -79.58984,\n 6: 0.00005,\n 7: -79.55933,\n ...\n }\n '

In [19]:
def tokenize_example(example, sos_token, eos_token):
    input_tokens = [str(num) for num in example["input"]]
    output_tokens = [str(num) for num in example["output"]]
    input_tokens = [sos_token] + input_tokens + [eos_token]
    output_tokens = [sos_token] + output_tokens + [eos_token]
    return {"input_tokens": input_tokens, "output_tokens": output_tokens}

In [20]:
sos_token = "<sos>"
eos_token = "<eos>"

fn_kwargs = {
    "sos_token": sos_token, 
    "eos_token": eos_token
}

In [21]:
train_data = Dataset.from_list(train_data)
valid_data = Dataset.from_list(valid_data)
test_data = Dataset.from_list(test_data)

In [22]:
def tokenize_example(example, sos_token, eos_token):
    input_tokens = [str(num) for num in example["input"]]
    output_tokens = [str(num) for num in example["output"]]
    input_tokens = [sos_token] + input_tokens + [eos_token]
    output_tokens = [sos_token] + output_tokens + [eos_token]
    return {"input_tokens": input_tokens, "output_tokens": output_tokens}

In [23]:
sos_token = "<sos>"
eos_token = "<eos>"

fn_kwargs = {
    "sos_token": sos_token, 
    "eos_token": eos_token
}

In [24]:
train_data = train_data.map(lambda x: tokenize_example(x, **fn_kwargs))
valid_data = valid_data.map(lambda x: tokenize_example(x, **fn_kwargs))
test_data = test_data.map(lambda x: tokenize_example(x, **fn_kwargs))

Map:   0%|          | 0/21 [00:00<?, ? examples/s]

Map:   0%|          | 0/2 [00:00<?, ? examples/s]

Map:   0%|          | 0/4 [00:00<?, ? examples/s]

In [25]:
train_data[0]['output_tokens']

['<sos>',
 '0.025',
 '-82.1228',
 '0.035',
 '54.01611',
 '0.1',
 '37.20093',
 '0.3',
 '9.61304',
 '0.45',
 '-79.07104',
 '1.9999',
 '-82.51953',
 '<eos>']

In [26]:
from collections import Counter, defaultdict

class Vocab:
    def __init__(self, token_to_index, unk_token="<unk>"):
        self.token_to_index = token_to_index
        self.index_to_token = {idx:token for token,idx in token_to_index.items()}
        self.unk_token = unk_token
        self.unk_index = token_to_index[unk_token]

    def __len__(self):
        return len(self.token_to_index)

    def __getitem__(self, token):
        return self.token_to_index.get(token, self.unk_index)

    def token_to_idx(self, token):
        return self.__getitem__(token)

    def idx_to_token(self, index):
        return self.index_to_token.get(index, self.unk_token)

    def set_default_token(self, index):
        if index == self.unk_index:
            return

        current_token = self.index_to_token.get(index, None)

        if self.unk_token in self.token_to_index and current_token:
            self.token_to_index[self.unk_token], self.token_to_index[current_token] = index, self.unk_index

        if self.unk_index in self.index_to_token and index in self.index_to_token:
            self.index_to_token[self.unk_index], self.index_to_token[index] = self.index_to_token[index], self.index_to_token[self.unk_index]

        self.unk_index = index

    def get_stoi(self):
        return self.token_to_index

    def get_itos(self):
        return self.index_to_token

    def lookup_tokens(self, indices):
        if torch.is_tensor(indices):
            indices = indices.tolist()

        return [self.idx_to_token(index) for index in indices]

    def lookup_indices(self, tokens):
        return [self.token_to_idx(token) for token in tokens]





In [27]:
def build_vocab_from_iterator(iterator, min_freq=1, specials=None):
    counter = Counter()
    for token in iterator:
        counter.update(token)

    token_to_index = {}
    if specials:
        for idx, token in enumerate(specials):
            token_to_index[token] = idx

    for token, freq in counter.items():
        if freq >= min_freq and token not in token_to_index:
            token_to_index[token] = len(token_to_index)

    unk_token = specials[0] if specials else "<unk>"
    if unk_token not in token_to_index:
        token_to_index[unk_token] = len(token_to_index)

    return Vocab(token_to_index, unk_token=unk_token)


tokens = [['<sos>',
  '0.02',
  '-79.40674',
  '0.036',
  '42.51099',
  '0.08',
  '22.33887',
  '0.18',
  '-1.58691',
  '0.25',
  '-77.63672',
  '1.99995',
  '-79.49829',
  '<eos>']]
min_freq = 1
specials = ["<unk>", "<pad>", "<sos>", "<eos>"]
vocab = build_vocab_from_iterator(tokens, min_freq, specials)
print(vocab.get_stoi())
print(vocab.idx_to_token(6))

    

{'<unk>': 0, '<pad>': 1, '<sos>': 2, '<eos>': 3, '0.02': 4, '-79.40674': 5, '0.036': 6, '42.51099': 7, '0.08': 8, '22.33887': 9, '0.18': 10, '-1.58691': 11, '0.25': 12, '-77.63672': 13, '1.99995': 14, '-79.49829': 15}
0.036


In [28]:
train_data["output_tokens"]

[['<sos>',
  '0.025',
  '-82.1228',
  '0.035',
  '54.01611',
  '0.1',
  '37.20093',
  '0.3',
  '9.61304',
  '0.45',
  '-79.07104',
  '1.9999',
  '-82.51953',
  '<eos>'],
 ['<sos>',
  '0.025',
  '-82.1228',
  '0.035',
  '54.01611',
  '0.1',
  '37.20093',
  '0.3',
  '9.61304',
  '0.45',
  '-79.07104',
  '1.9999',
  '-82.51953',
  '<eos>'],
 ['<sos>',
  '0.025',
  '-80.10864',
  '0.036',
  '52.55127',
  '0.06',
  '38.87939',
  '0.23',
  '11.38306',
  '0.35',
  '-78.27759',
  '1.9999',
  '-80.04761',
  '<eos>'],
 ['<sos>',
  '0.025',
  '-80.3833',
  '0.0345',
  '55.29785',
  '0.045',
  '38.48267',
  '0.14',
  '12.29858',
  '0.23',
  '-78.61328',
  '1.9999',
  '-80.41382',
  '<eos>'],
 ['<sos>',
  '0.025',
  '-80.01709',
  '0.036',
  '52.82593',
  '0.08',
  '37.07886',
  '0.27',
  '9.3689',
  '0.39',
  '-78.49121',
  '1.9999',
  '-80.29175',
  '<eos>'],
 ['<sos>',
  '0.025',
  '-80.26123',
  '0.034',
  '56.39648',
  '0.08',
  '38.05542',
  '0.28',
  '17.82227',
  '0.45',
  '-79.31519',
  '1

In [29]:
min_freq = 1
unk_token = "<unk>"
pad_token = "<pad>"

special_tokens = [
    unk_token,
    pad_token,
    sos_token,
    eos_token,
]

input_vocab = build_vocab_from_iterator(
    train_data["input_tokens"],
    min_freq=min_freq,
    specials=special_tokens,
)

output_vocab = build_vocab_from_iterator(
    train_data["output_tokens"],
    min_freq=min_freq,
    specials=special_tokens,
)

In [30]:
print(len(input_vocab))
print(len(output_vocab))

44761
157


In [31]:
assert input_vocab[unk_token] == output_vocab[unk_token]
assert output_vocab[pad_token] == input_vocab[pad_token]

unk_index = input_vocab[unk_token]
pad_index = output_vocab[pad_token]

In [32]:
input_vocab.set_default_token(unk_index)
output_vocab.set_default_token(unk_index)

In [33]:
def numericalize_example(example, input_vocab, output_vocab):
    input_ids = input_vocab.lookup_indices(example["input_tokens"])
    output_ids = output_vocab.lookup_indices(example["output_tokens"])
    return {"input_ids": input_ids, "output_ids": output_ids}

In [34]:
fn_kwargs = {"input_vocab": input_vocab, "output_vocab": output_vocab}

train_data = train_data.map(numericalize_example, fn_kwargs=fn_kwargs)
valid_data = valid_data.map(numericalize_example, fn_kwargs=fn_kwargs)
test_data = test_data.map(numericalize_example, fn_kwargs=fn_kwargs)

Map:   0%|          | 0/21 [00:00<?, ? examples/s]

Map:   0%|          | 0/2 [00:00<?, ? examples/s]

Map:   0%|          | 0/4 [00:00<?, ? examples/s]

In [35]:
data_type = "torch"
format_columns = ["input_ids", "output_ids"]

train_data = train_data.with_format(
    type=data_type, columns=format_columns, output_all_columns=True
)

valid_data = valid_data.with_format(
    type=data_type, columns=format_columns, output_all_columns=True
)

test_data = test_data.with_format(
    type=data_type, columns=format_columns, output_all_columns=True
)



In [36]:
def get_collate_fn(pad_index):
    def collate_fn(batch):
        batch_input_ids = [example["input_ids"] for example in batch]
        batch_output_ids = [example["output_ids"] for example in batch]
        batch_input_ids = nn.utils.rnn.pad_sequence(batch_input_ids, padding_value=pad_index)
        batch_output_ids = nn.utils.rnn.pad_sequence(batch_output_ids, padding_value=pad_index)
        batch = {
            "input_ids": batch_input_ids,
            "output_ids": batch_output_ids
        }
        return batch
    return collate_fn

In [37]:
def get_data_loader(dataset, batch_size, pad_index, shuffle=False):
    collate_fn = get_collate_fn(pad_index)
    data_loader = torch.utils.data.DataLoader(
        dataset=dataset,
        batch_size=batch_size,
        collate_fn=collate_fn,
        shuffle=shuffle,
    )
    return data_loader

In [38]:
batch_size = 5

train_data_loader = get_data_loader(train_data, batch_size, pad_index, shuffle=True)
valid_data_loader = get_data_loader(valid_data, batch_size, pad_index)
test_data_loader = get_data_loader(test_data, batch_size, pad_index)


                                    

In [39]:
class Encoder(nn.Module):
    def __init__(
        self, input_dim, embedding_dim, encoder_hidden_dim, decoder_hidden_dim, dropout
    ):
        super().__init__()
        self.embedding = nn.Embedding(input_dim, embedding_dim)
        self.rnn = nn.GRU(embedding_dim, encoder_hidden_dim, bidirectional=True)
        self.fc = nn.Linear(encoder_hidden_dim * 2, decoder_hidden_dim)
        self.dropout = nn.Dropout(dropout)

    def forward(self, src):
        embedded = self.dropout(self.embedding(src))
        outputs, hidden = self.rnn(embedded)
        hidden = torch.tanh(
            self.fc(torch.cat((hidden[-2, :, :], hidden[-1, :, :]), dim=1))
        )
        return outputs, hidden

In [40]:
class Attention(nn.Module):
    def __init__(self, encoder_hidden_dim, decoder_hidden_dim):
        super().__init__()
        self.attn_fc = nn.Linear((encoder_hidden_dim * 2) + decoder_hidden_dim, decoder_hidden_dim)
        self.v_fc = nn.Linear(decoder_hidden_dim, 1, bias=False)

    def forward(self, hidden, encoder_outputs):
        src_length = encoder_outputs.shape[0]
        hidden = hidden.unsqueeze(1).repeat(1, src_length, 1)
        encoder_outputs = encoder_outputs.permute(1, 0, 2)
        energy = torch.tanh(self.attn_fc(torch.cat((hidden, encoder_outputs), dim=2)))
        attention = self.v_fc(energy).squeeze(2)
        return torch.softmax(attention, dim=1)
        
            



In [41]:
class Decoder(nn.Module):
    def __init__(
        self,
        output_dim,
        embedding_dim,
        encoder_hidden_dim,
        decoder_hidden_dim,
        dropout,
        attention,
    ):
        super().__init__()
        self.output_dim = output_dim
        self.attention = attention
        self.embedding = nn.Embedding(output_dim, embedding_dim)
        self.rnn = nn.GRU(
            (encoder_hidden_dim * 2) + embedding_dim, decoder_hidden_dim
        )
        self.fc_out = nn.Linear(
            (encoder_hidden_dim * 2) + decoder_hidden_dim + embedding_dim, output_dim
        )
        self.dropout = nn.Dropout(dropout)

    def forward(self, input, hidden, encoder_outputs):
        input = input.unsqueeze(0)
        embedded = self.dropout(self.embedding(input))
        a = self.attention(hidden, encoder_outputs)
        a = a.unsqueeze(1)
        encoder_outputs = encoder_outputs.permute(1, 0, 2)
        weighted = torch.bmm(a, encoder_outputs)
        weighted = weighted.permute(1, 0, 2)
        output, hidden = self.rnn(torch.cat((embedded, weighted), dim=2), hidden.unsqueeze(0))
        output = output.squeeze(0)
        weighted = weighted.squeeze(0)
        embedded = embedded.squeeze(0)
        prediction = self.fc_out(torch.cat((output, weighted, embedded), dim=1))
        return prediction, hidden.squeeze(0), a.squeeze(1)
        
        

In [42]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.device = device

    def forward(self, src, trg, teacher_forcing_ratio):
        batch_size = src.shape[1]
        trg_length = trg.shape[0]
        trg_vocab_size = self.decoder.output_dim
        outputs = torch.zeros(trg_length, batch_size, trg_vocab_size).to(self.device)
        encoder_outputs, hidden = self.encoder(src)
        input = trg[0, :]
        for t in range(1, trg_length):
            output, hidden, _ = self.decoder(input, hidden, encoder_outputs)
            outputs[t] = output
            teacher_force = random.random() < teacher_forcing_ratio
            top1 = output.argmax(1)
            input = trg[t] if teacher_force else top1
        return outputs

In [43]:
input_dim = len(input_vocab)
output_dim = len(output_vocab)
encoder_embedding_dim = 256
decoder_embedding_dim = 256
encoder_hidden_dim = 512
decoder_hidden_dim = 512
encoder_dropout = 0.5
decoder_dropout = 0.5
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

attention = Attention(encoder_hidden_dim, decoder_hidden_dim)

encoder = Encoder(
    input_dim,
    encoder_embedding_dim,
    encoder_hidden_dim,
    decoder_hidden_dim,
    encoder_dropout,
)

decoder = Decoder(
    output_dim,
    decoder_embedding_dim,
    encoder_hidden_dim,
    decoder_hidden_dim,
    decoder_dropout,
    attention,
)

model = Seq2Seq(encoder, decoder, device).to(device)

In [44]:
def init_weights(m):
    for name, param in m.named_parameters():
        if "weight" in name:
            nn.init.normal_(param.data, mean=0, std=0.01)
        else:
            nn.init.constant_(param.data, 0)


model.apply(init_weights)

Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(44761, 256)
    (rnn): GRU(256, 512, bidirectional=True)
    (fc): Linear(in_features=1024, out_features=512, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (decoder): Decoder(
    (attention): Attention(
      (attn_fc): Linear(in_features=1536, out_features=512, bias=True)
      (v_fc): Linear(in_features=512, out_features=1, bias=False)
    )
    (embedding): Embedding(157, 256)
    (rnn): GRU(1280, 512)
    (fc_out): Linear(in_features=1792, out_features=157, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
)

In [45]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)


print(f"The model has {count_parameters(model):,} trainable parameters")

The model has 18,213,789 trainable parameters


In [46]:
optimizer = optim.Adam(model.parameters())

In [47]:
criterion = nn.CrossEntropyLoss(ignore_index=pad_index)

In [48]:
def train_fn(
    model, data_loader, optimizer, criterion, clip, teacher_forcing_ratio, device
):
    model.train()
    epoch_loss = 0
    for i, batch in enumerate(data_loader):
        src = batch["input_ids"].to(device)
        trg = batch["output_ids"].to(device)
        # src = [src length, batch size]
        # trg = [trg length, batch size]
        optimizer.zero_grad()
        output = model(src, trg, teacher_forcing_ratio)
        # output = [trg length, batch size, trg vocab size]
        output_dim = output.shape[-1]
        output = output[1:].view(-1, output_dim)
        # output = [(trg length - 1) * batch size, trg vocab size]
        trg = trg[1:].view(-1)
        # trg = [(trg length - 1) * batch size]
        loss = criterion(output, trg)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        optimizer.step()
        epoch_loss += loss.item()
    return epoch_loss / len(data_loader)

In [49]:
def evaluate_fn(model, data_loader, criterion, device):
    model.eval()
    epoch_loss = 0
    with torch.no_grad():
        for i, batch in enumerate(data_loader):
            src = batch["input_ids"].to(device)
            trg = batch["output_ids"].to(device)
            # src = [src length, batch size]
            # trg = [trg length, batch size]
            output = model(src, trg, 0)  # turn off teacher forcing
            # output = [trg length, batch size, trg vocab size]
            output_dim = output.shape[-1]
            output = output[1:].view(-1, output_dim)
            # output = [(trg length - 1) * batch size, trg vocab size]
            trg = trg[1:].view(-1)
            # trg = [(trg length - 1) * batch size]
            loss = criterion(output, trg)
            epoch_loss += loss.item()
    return epoch_loss / len(data_loader)

In [ ]:
n_epochs = 10
clip = 1.0
teacher_forcing_ratio = 0.5

best_valid_loss = float("inf")

for epoch in tqdm.tqdm(range(n_epochs)):
    train_loss = train_fn(
        model,
        train_data_loader,
        optimizer,
        criterion,
        clip,
        teacher_forcing_ratio,
        device,
    )
    valid_loss = evaluate_fn(
        model,
        valid_data_loader,
        criterion,
        device,
    )
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), "ecg-s2s-model.pt")
    
    print(f"\tTrain Loss: {train_loss:7.3f} | Train PPL: {np.exp(train_loss):7.3f}")
    print(f"\tValid Loss: {valid_loss:7.3f} | Valid PPL: {np.exp(valid_loss):7.3f}")



  0%|                                                    | 0/10 [00:00<?, ?it/s]

In [ ]:
model.load_state_dict(torch.load("ecg-s2s-model.pt"))

test_loss = evaluate_fn(model, test_data_loader, criterion, device)

print(f"| Test Loss: {test_loss:.3f} | Test PPL: {np.exp(test_loss):7.3f} |")

In [ ]:
def classify_ecg(
    data,
    model,
    output_vocab, 
    input_vocab,
    sos_token,
    eos_token,
    device,
    max_output_length=25,
):
    model.eval()
    with torch.no_grad():
        # check instance of data
        # convert to list of numbers
        if isinstance(data[0], str):
            input_tokens = [token for token in data]
        else:
            input_tokens = [str(num) for num in data]
        input_tokens = [sos_token] + input_tokens + [eos_token]
        ids = input_vocab.lookup_indices(input_tokens)
        tensor = torch.LongTensor(ids).unsqueeze(-1).to(device)
        encoder_outputs, hidden = model.encoder(tensor)
        inputs = output_vocab.lookup_indices([sos_token])
        attentions = torch.zeros(max_output_length, 1, len(ids))
        for i in range(max_output_length):
            input_tensor = torch.LongTensor([inputs[-1]]).to(device)
            output, hidden, attention = model.decoder(
                input_tensor, hidden, encoder_outputs
            )
            attentions[i] = attention
            predicted_token = output.argmax(-1).item()
            inputs.append(predicted_token)
            if predicted_token == output_vocab[eos_token]:
                break
        output_tokens = output_vocab.lookup_tokens(inputs)
    return output_tokens, input_tokens, attentions[: len(output_tokens) - 1]



In [ ]:
data = dataset[0]["input"]
expected = dataset[0]["output"]

expected


In [ ]:
output, data_tokens, attention = classify_ecg(
    data,
    model, 
    output_vocab,
    input_vocab,
    sos_token,
    eos_token,
    device,
)

In [ ]:
output, expected

In [ ]:
translations = [
    classify_ecg(
        example["input"],
        model,
        output_vocab,
        input_vocab,
        sos_token,
        eos_token,
        device,
    )[0]
    for example in tqdm.tqdm(test_data)
]

In [ ]:
bleu = evaluate.load("bleu")

In [ ]:
translations = [output]
predictions = [" ".join(translation[1:-1]) for translation in translations]

references = [[example["input"]] for example in test_data]



In [ ]:
def get_tokenizer_fn():
    def tokenizer_fn(s):
        if isinstance(s[0], str):
            tokens = [token for token in s]
        else:
            tokens = [str(num) for num in s]
        return tokens

    return tokenizer_fn

In [ ]:
tokenizer_fn = get_tokenizer_fn()

In [ ]:
results = bleu.compute(
    predictions=predictions, references=references, tokenizer=tokenizer_fn
)

In [ ]:
results